In [1]:
#!/usr/bin/env python3

import sys
import matplotlib.pyplot as plt
import os

from collections import namedtuple
import util as cm
import cv2
import time
import numpy as np

In [2]:
def perform_skeleton_tracking_alg(skeletrack, color_image, depth_image, depth_intrinsic, baseline_done): 
    
    depth_scaled = depth_image * 0.0010000000474974513
    
    # perform inference and update the tracking id
    skeletons = skeletrack.track_skeletons(color_image)
    
    # render the skeletons on top of the acquired image and display it
    color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
    depth_image_color = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)
    
    # render leg position on image 
    if len(skeletons) > 0: 
        try: 
            detect_legs_and_angle(depth_image_color, skeletons[0])
        except: 
            None
        try:    
            if not baseline_done: 
                original_x = np.mean([skeletons[0].joints[5][0], skeletons[0].joints[2][0]])
            computed_side_movement(depth_image_color, skeletons[0], original_x)
        except Exception as e: 
            print('error in side to side movement')
            print(e)
            
        try: 
            if not baseline_done: 
                x_ls, y_ls = int(skeletons[0].joints[5][0]), int(skeletons[0].joints[5][1])
                x_rs, y_rs = int(skeletons[0].joints[2][0]), int(skeletons[0].joints[2][1])
                x_mid, y_mid = int(skeletons[0].joints[1][0]), int(skeletons[0].joints[1][1])
                z_ls, z_rs, z_mid = depth_scaled[y_ls, x_ls], depth_scaled[y_rs, x_rs], depth_scaled[y_mid, x_mid]
                ref_z = np.mean([z_ls, z_rs, z_mid])
            compute_rocking(depth_scaled, depth_image_color, skeletons[0], ref_z)
        except Exception as e: 
            print('error in rocking')
            print(e)
            
    # obtain region of interest for chest using skeleton points 
    curr_y_ls, curr_y_rs = int(skeletons[0].joints[5][1]), int(skeletons[0].joints[2][1])
    curr_x_la, curr_x_ra = int(skeletons[0].joints[6][0]), int(skeletons[0].joints[3][0])
    curr_x_lw, curr_y_lw = int(skeletons[0].joints[11][0]), int(skeletons[0].joints[11][1])
    curr_x_rw, curr_y_rw = int(skeletons[0].joints[8][0]), int(skeletons[0].joints[8][1])
    
    chest_y0 = max(curr_y_ls, curr_y_rs)
    bottom_y = min(curr_y_lw, curr_y_rw)
    chest_y1 = (chest_y0 + bottom_y) // 2
    
    chest_roi = depth_scaled[chest_y0:bottom_y,curr_x_ra:curr_x_la]
    depth_imgs.append(chest_roi)
    chest_dist = np.mean(chest_roi)
    distances.append(chest_dist)
            
    cm.render_result(skeletons, depth_image_color, joint_confidence)
    render_ids_3d(depth_image_color, skeletons, depth.as_depth_frame(), depth_intrinsic, joint_confidence)
    
    cv2.imshow(window_name, depth_image_color)
    
    return

def render_ids_3d(render_image, skeletons_2d, depth_map, depth_intrinsic, joint_confidence):
    thickness = 1
    text_color = (255, 255, 255)
    rows, cols, channel = render_image.shape[:3]
    distance_kernel_size = 5
    # calculate 3D keypoints and display them
    for skeleton_index in range(len(skeletons_2d)):
        skeleton_2D = skeletons_2d[skeleton_index]
        joints_2D = skeleton_2D.joints
        did_once = False
        for joint_index in range(len(joints_2D)):
            if did_once == False:
                cv2.putText(
                    render_image,
                    "id: " + str(skeleton_2D.id),
                    (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y - 30)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.55,
                    text_color,
                    thickness,
                )
                did_once = True
            # check if the joint was detected and has valid coordinate
            if skeleton_2D.confidences[joint_index] > joint_confidence:
                distance_in_kernel = []
                low_bound_x = max(
                    0,
                    int(
                        joints_2D[joint_index].x - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_x = min(
                    cols - 1,
                    int(joints_2D[joint_index].x + math.ceil(distance_kernel_size / 2)),
                )
                low_bound_y = max(
                    0,
                    int(
                        joints_2D[joint_index].y - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_y = min(
                    rows - 1,
                    int(joints_2D[joint_index].y + math.ceil(distance_kernel_size / 2)),
                )
                for x in range(low_bound_x, upper_bound_x):
                    for y in range(low_bound_y, upper_bound_y):
                        distance_in_kernel.append(depth_map.get_distance(x, y))
                median_distance = np.percentile(np.array(distance_in_kernel), 50)
                depth_pixel = [
                    int(joints_2D[joint_index].x),
                    int(joints_2D[joint_index].y),
                ]
                if median_distance >= 0.3:
                    point_3d = rs.rs2_deproject_pixel_to_point(
                        depth_intrinsic, depth_pixel, median_distance
                    )
                    point_3d = np.round([float(i) for i in point_3d], 3)
                    point_str = [str(x) for x in point_3d]
                    cv2.putText(
                        render_image,
                        str(point_3d),
                        (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y)),
                        cv2.FONT_HERSHEY_DUPLEX,
                        0.4,
                        text_color,
                        thickness,
                    )
                    
def compute_orientation(img, display=False):
    
    # apply threshold
    thresh = threshold_otsu(img)
    bw = closing(img > thresh, square(3))

    label_img = label(bw)
    props = regionprops(label_img)

    # largest img
    largest_index = np.argmax([p.area for p in props])
    prop = props[largest_index]
    
    if display: 

        plt.figure()
        plt.imshow(prop.image)

        x0 = prop['Centroid'][1]
        y0 = prop['Centroid'][0]
        x2 = x0 - math.sin(prop['Orientation']) * 0.9 * prop['MinorAxisLength']
        y2 = y0 - math.cos(prop['Orientation']) * 0.9 * prop['MinorAxisLength']

        plt.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
        plt.plot(x0, y0, '.g', markersize=15)

        minr, minc, maxr, maxc = prop['BoundingBox']
        bx = (minc, maxc, maxc, minc, minc)
        by = (minr, minr, maxr, maxr, minr)
        
        plt.plot(bx, by, '-b', linewidth=2.5)
        plt.show()
        
    return prop.orientation

def detect_legs(img, skeleton): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
    
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la, x_la-30:x_la+30] = img[y_lk:y_la, x_la-30:x_la+30]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra, y_ra-30:y_ra+30] = img[y_rk:y_ra, x_ra-30:x_ra+30]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # find contours for each leg 
    x_l, y_l, w_l, h_l = cv2.boundingRect(thresh_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(thresh_right)

    final_img = cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), (36,255,12), 2)
    final_img = cv2.rectangle(final_img, (x_r, y_r), (x_r + w_r, y_r + h_r), (36,255,12), 2)

    # display
    cv2.putText(final_img, 'left leg', (x_l, y_l-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    cv2.putText(final_img, 'right leg', (x_r, y_r-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow('image', final_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def detect_legs_and_angle(img, skeleton, display=False): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
        
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la-15, x_la-70:x_la+70] = img[y_lk:y_la-15, x_la-70:x_la+70]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra-15, x_ra-70:x_ra+70] = img[y_rk:y_ra-15, x_ra-70:x_ra+70]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # obtain contour
    cntrs_left = cv2.findContours(thresh_left, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_left = cntrs_left[0] if len(cntrs_left) == 2 else cntrs_left[1]
    cntrs_left = max(cntrs_left, key=cv2.contourArea)
    cntrs_right = cv2.findContours(thresh_right, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_right = cntrs_right[0] if len(cntrs_right) == 2 else cntrs_right[1]
    cntrs_right = max(cntrs_right, key=cv2.contourArea)

    # get bounding box for legs
    x_l, y_l, w_l, h_l = cv2.boundingRect(cntrs_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(cntrs_right)

    # fit ellipsoid to find orientation 
    ellipse_left, ellipse_right = cv2.fitEllipse(cntrs_left), cv2.fitEllipse(cntrs_right)

    _, angle_l = get_angle_orienation(img, ellipse_left)
    _, angle_r = get_angle_orienation(img, ellipse_right)

    # display 
    if angle_l >= 80 and angle_l <= 101: 
        color_l = (36,255,12)
        status_l = True
    else: 
        color_l = (255,12,36)
        status_l = False


    if angle_r >= 80 and angle_r <= 101:
        color_r = (36,255,12)
        status_r = True
    else: 
        color_r = (255,12,36)
        status_r = False

    cv2.putText(img, str(round(angle_l, 1)), (x_l, y_l-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_l, 2)
    cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), color_l, 2)

    cv2.putText(img, str(round(angle_r, 1)), (x_r, y_r-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_r, 2)
    cv2.rectangle(img, (x_r, y_r), (x_r + w_r, y_r + h_r), color_r, 2)

    if status_r and status_l: 
        cv2.putText(img, 'Legs Good Position', (img.shape[1]//2+200, img.shape[0]-25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 3)
    else: 
        cv2.putText(img, 'Straighten Legs', (img.shape[1]//2+200, img.shape[0]-25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 3)
    
    if display: 
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
def get_angle_orienation(img, ellipse, draw_line=False): 
    
    result = None
    (xc, yc), (d1, d2), angle = ellipse
    
    # draw orienation line
    rmajor = max(d1, d2) / 2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    xtop = xc + math.cos(math.radians(angle))*rmajor
    ytop = yc + math.sin(math.radians(angle))*rmajor
    xbot = xc + math.cos(math.radians(angle+180))*rmajor
    ybot = yc + math.sin(math.radians(angle+180))*rmajor
    
    if draw_line:
        result = cv2.line(img, (int(xtop),int(ytop)), (int(xbot),int(ybot)), (0, 0, 255), 3)
    
    return result, angle

def compute_rocking(depth_img, colorized, skeleton, prev_z): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    x_mid, y_mid = int(skeleton.joints[1][0]), int(skeleton.joints[1][1])
    z_ls, z_rs, z_mid = depth_img[y_ls, x_ls], depth_img[y_rs, x_rs], depth_img[y_mid, x_mid]
        
    # compare to last z location 
    avg_position = np.mean([z_ls, z_rs, z_mid])
    # display bounding box and status of rocking
    if abs(prev_z - avg_position) > .05: 
        cv2.putText(colorized, 'Rocking Detected', (x_ls+5, y_rs-70), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(colorized, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        
def computed_side_movement(img, skeleton, prev_x): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    
    # compare to previous x value and display 
    avg_position = np.mean([x_ls, x_rs])
    if abs(prev_x - avg_position) > 5: 
        cv2.putText(img, 'Side Movement Detected', (x_ls+5, y_rs-40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(img, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        
def convert_depth_frame_to_pointcloud(depth_image, camera_intrinsics ):
    """
    Convert the depthmap to a 3D point cloud
    Parameters:
    -----------
    depth_frame : rs.frame() The depth_frame containing the depth map
    camera_intrinsics : The intrinsic values of the imager in whose coordinate system the depth_frame is computed
    Return:
    ----------
    x : array
        The x values of the pointcloud in meters
    y : array
        The y values of the pointcloud in meters
    z : array
        The z values of the pointcloud in meters
    """

    [height, width] = depth_image.shape

    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - camera_intrinsics.ppx)/camera_intrinsics.fx
    y = (v.flatten() - camera_intrinsics.ppy)/camera_intrinsics.fy

    z = depth_image.flatten() / 1000;
    x = np.multiply(x,z)
    y = np.multiply(y,z)

    x = x[np.nonzero(z)]
    y = y[np.nonzero(z)]
    z = z[np.nonzero(z)]

    return x, y, z

In [ ]:
# License: Apache 2.0. See LICENSE file in root directory.
# Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

"""
OpenGL Pointcloud viewer with http://pyglet.org
Usage:
------
Mouse:
    Drag with left button to rotate around pivot (thick small axes),
    with right button to translate and the wheel to zoom.
Keyboard:
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [x]     Toggle point distance attenuation
    [c]     Toggle color source
    [l]     Toggle lighting
    [f]     Toggle depth post-processing
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q/ESC] Quit
Notes:
------
Using deprecated OpenGL (FFP lighting, matrix stack...) however, draw calls 
are kept low with pyglet.graphics.* which uses glDrawArrays internally.
Normals calculation is done with numpy on CPU which is rather slow, should really
be done with shaders but was omitted for several reasons - brevity, for lowering
dependencies (pyglet doesn't ship with shader support & recommends pyshaders)
and for reference.
"""

import math
import ctypes
import pyglet
import pyglet.gl as gl
import numpy as np
import pyrealsense2 as rs

def get_clipped_pointcloud(pointcloud, boundary):

    assert (pointcloud.shape[0]>=2)
    
    pointcloud = pointcloud[:,np.logical_and(pointcloud[0,:]<boundary[1], pointcloud[0,:]>boundary[0])]
    pointcloud = pointcloud[:,np.logical_and(pointcloud[1,:]<boundary[3], pointcloud[1,:]>boundary[2])]
    
    return pointcloud


# https://stackoverflow.com/a/6802723
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])


class AppState:

    def __init__(self, *args, **kwargs):
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, 1], np.float32)
        self.distance = 2
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 0
        self.scale = True
        self.attenuation = False
        self.color = True
        self.lighting = False
        self.postprocessing = False
        self.baseline_done = False

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, 1

    @property
    def rotation(self):
        Rx = rotation_matrix((1, 0, 0), math.radians(-self.pitch))
        Ry = rotation_matrix((0, 1, 0), math.radians(-self.yaw))
        return np.dot(Ry, Rx).astype(np.float32)

state = AppState()

# Configure streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_device('020122061309')
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
other_stream, other_format = rs.stream.color, rs.format.rgb8
config.enable_stream(other_stream, 640, 480, other_format, 30)

# Start streaming
pipeline.start(config)
profile = pipeline.get_active_profile()

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

# Processing blocks
pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()
filters = [rs.disparity_transform(),
           rs.spatial_filter(),
           rs.temporal_filter(),
           rs.disparity_transform(False)]

# pyglet
window = pyglet.window.Window(
    config=gl.Config(
        double_buffer=True,
        samples=8  # MSAA
    ),
    resizable=True, vsync=True)
keys = pyglet.window.key.KeyStateHandler()
window.push_handlers(keys)

def convert_fmt(fmt):
    """rs.format to pyglet format string"""
    return {
        rs.format.rgb8: 'RGB',
        rs.format.bgr8: 'BGR',
        rs.format.rgba8: 'RGBA',
        rs.format.bgra8: 'BGRA',
        rs.format.y8: 'L',
    }[fmt]

# Create a VertexList to hold pointcloud data
# Will pre-allocates memory according to the attributes below
vertex_list = pyglet.graphics.vertex_list(
    w * h, 'v3f/stream', 't2f/stream', 'n3f/stream')
# Create and allocate memory for our color data
other_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))
image_data = pyglet.image.ImageData(w, h, convert_fmt(
    other_profile.format()), (gl.GLubyte * (w * h * 3))())

fps_display = pyglet.window.FPSDisplay(window)


@window.event
def on_mouse_drag(x, y, dx, dy, buttons, modifiers):
    w, h = map(float, window.get_size())

    if buttons & pyglet.window.mouse.LEFT:
        state.yaw -= dx * 0.5
        state.pitch -= dy * 0.5

    if buttons & pyglet.window.mouse.RIGHT:
        dp = np.array((dx / w, -dy / h, 0), np.float32)
        state.translation += np.dot(state.rotation, dp)

    if buttons & pyglet.window.mouse.MIDDLE:
        dz = dy * 0.01
        state.translation -= (0, 0, dz)
        state.distance -= dz


def handle_mouse_btns(x, y, button, modifiers):
    state.mouse_btns[0] ^= (button & pyglet.window.mouse.LEFT)
    state.mouse_btns[1] ^= (button & pyglet.window.mouse.RIGHT)
    state.mouse_btns[2] ^= (button & pyglet.window.mouse.MIDDLE)


window.on_mouse_press = window.on_mouse_release = handle_mouse_btns


@window.event
def on_mouse_scroll(x, y, scroll_x, scroll_y):
    dz = scroll_y * 0.1
    state.translation -= (0, 0, dz)
    state.distance -= dz


def on_key_press(symbol, modifiers):
    if symbol == pyglet.window.key.R:
        state.reset()

    if symbol == pyglet.window.key.P:
        state.paused ^= True

    if symbol == pyglet.window.key.D:
        state.decimate = (state.decimate + 1) % 3
        decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)

    if symbol == pyglet.window.key.C:
        state.color ^= True

    if symbol == pyglet.window.key.Z:
        state.scale ^= True

    if symbol == pyglet.window.key.X:
        state.attenuation ^= True

    if symbol == pyglet.window.key.L:
        state.lighting ^= True

    if symbol == pyglet.window.key.F:
        state.postprocessing ^= True

    if symbol == pyglet.window.key.S:
        pyglet.image.get_buffer_manager().get_color_buffer().save('out.png')

    if symbol == pyglet.window.key.Q:
        window.close()

window.push_handlers(on_key_press)

def axes(size=1, width=1):
    """draw 3d axes"""
    gl.glLineWidth(width)
    pyglet.graphics.draw(6, gl.GL_LINES,
                         ('v3f', (0, 0, 0, size, 0, 0,
                                  0, 0, 0, 0, size, 0,
                                  0, 0, 0, 0, 0, size)),
                         ('c3f', (1, 0, 0, 1, 0, 0,
                                  0, 1, 0, 0, 1, 0,
                                  0, 0, 1, 0, 0, 1,
                                  ))
                         )


def frustum(intrinsics):
    """draw camera's frustum"""
    w, h = intrinsics.width, intrinsics.height
    batch = pyglet.graphics.Batch()

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            batch.add(2, gl.GL_LINES, None, ('v3f', [0, 0, 0] + p))
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        batch.add(2, gl.GL_LINES, None, ('v3f', top_left + top_right))
        batch.add(2, gl.GL_LINES, None, ('v3f', top_right + bottom_right))
        batch.add(2, gl.GL_LINES, None, ('v3f', bottom_right + bottom_left))
        batch.add(2, gl.GL_LINES, None, ('v3f', bottom_left + top_left))

    batch.draw()


def grid(size=1, n=10, width=1):
    """draw a grid on xz plane"""
    gl.glLineWidth(width)
    s = size / float(n)
    s2 = 0.5 * size
    batch = pyglet.graphics.Batch()

    for i in range(0, n + 1):
        x = -s2 + i * s
        batch.add(2, gl.GL_LINES, None, ('v3f', (x, 0, -s2, x, 0, s2)))
    for i in range(0, n + 1):
        z = -s2 + i * s
        batch.add(2, gl.GL_LINES, None, ('v3f', (-s2, 0, z, s2, 0, z)))

    batch.draw()


@window.event
def on_draw():
    window.clear()

    gl.glEnable(gl.GL_DEPTH_TEST)
    gl.glEnable(gl.GL_LINE_SMOOTH)

    width, height = window.get_size()
    gl.glViewport(0, 0, width, height)

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.gluPerspective(60, width / float(height), 0.01, 20)

    gl.glMatrixMode(gl.GL_TEXTURE)
    gl.glLoadIdentity()
    # texcoords are [0..1] and relative to top-left pixel corner, add 0.5 to center
    gl.glTranslatef(0.5 / image_data.width, 0.5 / image_data.height, 0)
    image_texture = image_data.get_texture()
    # texture size may be increased by pyglet to a power of 2
    tw, th = image_texture.owner.width, image_texture.owner.height
    gl.glScalef(image_data.width / float(tw),
                image_data.height / float(th), 1)

    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()

    gl.gluLookAt(0, 0, 0, 0, 0, 1, 0, -1, 0)

    gl.glTranslatef(0, 0, state.distance)
    gl.glRotated(state.pitch, 1, 0, 0)
    gl.glRotated(state.yaw, 0, 1, 0)

    if any(state.mouse_btns):
        axes(0.1, 4)

    gl.glTranslatef(0, 0, -state.distance)
    gl.glTranslatef(*state.translation)

    gl.glColor3f(0.5, 0.5, 0.5)
    gl.glPushMatrix()
    gl.glTranslatef(0, 0.5, 0.5)
    grid()
    gl.glPopMatrix()

    psz = max(window.get_size()) / float(max(w, h)) if state.scale else 1
    gl.glPointSize(psz)
    distance = (0, 0, 1) if state.attenuation else (1, 0, 0)
    gl.glPointParameterfv(gl.GL_POINT_DISTANCE_ATTENUATION,
                          (gl.GLfloat * 3)(*distance))

    if state.lighting:
        ldir = [0.5, 0.5, 0.5]  # world-space lighting
        ldir = np.dot(state.rotation, (0, 0, 1))  # MeshLab style lighting
        ldir = list(ldir) + [0]  # w=0, directional light
        gl.glLightfv(gl.GL_LIGHT0, gl.GL_POSITION, (gl.GLfloat * 4)(*ldir))
        gl.glLightfv(gl.GL_LIGHT0, gl.GL_DIFFUSE,
                     (gl.GLfloat * 3)(1.0, 1.0, 1.0))
        gl.glLightfv(gl.GL_LIGHT0, gl.GL_AMBIENT,
                     (gl.GLfloat * 3)(0.75, 0.75, 0.75))
        gl.glEnable(gl.GL_LIGHT0)
        gl.glEnable(gl.GL_NORMALIZE)
        gl.glEnable(gl.GL_LIGHTING)

    gl.glColor3f(1, 1, 1)
    texture = image_data.get_texture()
    gl.glEnable(texture.target)
    gl.glBindTexture(texture.target, texture.id)
    gl.glTexParameteri(
        gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_NEAREST)

    # comment this to get round points with MSAA on
    gl.glEnable(gl.GL_POINT_SPRITE)

    if not state.scale and not state.attenuation:
        gl.glDisable(gl.GL_MULTISAMPLE)  # for true 1px points with MSAA on
    vertex_list.draw(gl.GL_POINTS)
    gl.glDisable(texture.target)
    if not state.scale and not state.attenuation:
        gl.glEnable(gl.GL_MULTISAMPLE)

    gl.glDisable(gl.GL_LIGHTING)

    gl.glColor3f(0.25, 0.25, 0.25)
    frustum(depth_intrinsics)
    axes()

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.glOrtho(0, width, 0, height, -1, 1)
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glMatrixMode(gl.GL_TEXTURE)
    gl.glLoadIdentity()
    gl.glDisable(gl.GL_DEPTH_TEST)

    fps_display.draw()

def run(dt):
    global w, h
    window.set_caption("RealSense (%dx%d) %dFPS (%.2fms) %s" %
                       (w, h, 0 if dt == 0 else 1.0 / dt, dt * 1000,
                        "PAUSED" if state.paused else ""))
    if state.paused:
        return
    
    success, frames = pipeline.try_wait_for_frames(timeout_ms=0)
    
    if not success:
        return

    depth_frame = frames.get_depth_frame().as_video_frame()
    other_frame = frames.first(other_stream).as_video_frame()

    depth_frame = decimate.process(depth_frame)

    if state.postprocessing:
        for f in filters:
            depth_frame = f.process(depth_frame)

    # Grab new intrinsics (may be changed by decimation)
    depth_intrinsics = rs.video_stream_profile(
        depth_frame.profile).get_intrinsics()
    w, h = depth_intrinsics.width, depth_intrinsics.height

    color_image = np.asanyarray(other_frame.get_data())

    colorized_depth = colorizer.colorize(depth_frame)
    depth_colormap = np.asanyarray(colorized_depth.get_data())

    if state.color:
        mapped_frame, color_source = other_frame, color_image
    else:
        mapped_frame, color_source = colorized_depth, depth_colormap

    points = pc.calculate(depth_frame)
    pc.map_to(mapped_frame)

    # handle color source or size change
    fmt = convert_fmt(mapped_frame.profile.format())
    global image_data
    if (image_data.format, image_data.pitch) != (fmt, color_source.strides[0]):
        empty = (gl.GLubyte * (w * h * 3))()
        image_data = pyglet.image.ImageData(w, h, fmt, empty)
    # copy image data to pyglet
    image_data.set_data(fmt, color_source.strides[0], color_source.ctypes.data)
    
    texcoords = np.asarray(points.get_texture_coordinates(2))
    verts = np.asarray(points.get_vertices(2))
        
    verts[np.where(verts[:,2]>2.4)] = [np.nan, np.nan, np.nan]
#     verts[np.where(verts[:,2]<=2.2)] = [np.nan, np.nan, np.nan]
#     verts = verts.reshape(h, w, 3)
#     texcoords = texcoords.reshape(h, w , 2)
    
#     verts = verts[220:330, 210:440, :]
#     texcoords = texcoords[220:330, 210:440]

    if len(vertex_list.vertices) != verts.size:
        vertex_list.resize(verts.size // 3)
        
        # need to reassign after resizing
        vertex_list.vertices = verts.ravel()
        vertex_list.tex_coords = texcoords.ravel()

    # copy our data to pre-allocated buffers, this is faster than assigning...
    # pyglet will take care of uploading to GPU
    def copy(dst, src):
        """copy numpy array to pyglet array"""
        # timeit was mostly inconclusive, favoring slice assignment for safety
        np.array(dst, copy=False)[:] = src.ravel()
        # ctypes.memmove(dst, src.ctypes.data, src.nbytes)

    copy(vertex_list.vertices, verts)
    copy(vertex_list.tex_coords, texcoords)

    if state.lighting:
        # compute normals
        dy, dx = np.gradient(verts, axis=(0, 1))
        n = np.cross(dx, dy)

        copy(vertex_list.normals, n)

    if keys[pyglet.window.key.E]:
        points.export_to_ply('./out.ply', mapped_frame)

pyglet.clock.schedule(run)
try:
    pyglet.app.run()
finally:
    pipeline.stop()